In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

path = "../input/imdb-project-sql/movies.sqlite"  #Insert path here

In [3]:
conn = sqlite3.connect(path)

In [4]:
table = pd.read_sql("""SELECT * FROM sqlite_master WHERE type='table';""", conn)
table

**Question 1: Can you get all data about movies?**

In [5]:
movies = pd.read_sql("""SELECT * FROM movies;""", conn)
movies.head()

In [6]:
movies.info()

In [7]:
movies.describe()

**Question 2: How do you get all data about directors?**


In [8]:
directors = pd.read_sql("""SELECT * FROM directors;""",conn)
directors

In [9]:
directors.info()

In [10]:
directors.describe()

In [11]:
movies.isnull().sum()

In [12]:
directors.isnull().sum()

In [13]:
cur = conn.cursor()

The description attribute of the cursor can be used to get just the names of the columns

In [14]:
cur.execute('SELECT * FROM movies').description

In [15]:
#cur.execute('ALTER TABLE movies DROP COLUMN tagline;')
#conn.commit()

**Question 3: Check how many movies are present in iMDB.**

In [16]:
count = cur.execute('SELECT COUNT (*) from movies;').fetchall()
count

**Question 4: Find these 3 directors: James Cameron ; Luc Besson ; John Woo**

In [17]:
dir3 = pd.DataFrame(
    cur.execute('SELECT * from directors where name == \'James Cameron\' or name ==\'Luc Besson\' or name ==\'John Woo\';').fetchall(), 
    columns = ['name','id','gender','uid','department'])
dir3

**Question 5: Find all directors with name starting with Steven.**

In [18]:
steven = pd.DataFrame(
    cur.execute('SELECT * from directors where name LIKE \'Steven%\';').fetchall(),
    columns = ['name','id','gender','uid','department']
)

steven

**Question 6: Count all male directors.**

In [19]:
female = cur.execute('SELECT COUNT(*) from directors where gender == 0;').fetchall()
female

**Question 7: Find the name of the 10 first women directors?**

In [20]:
first10 = pd.DataFrame(
    cur.execute('SELECT name FROM directors WHERE gender == 1 ORDER by id ASC limit 10 ;').fetchall(),
    columns = ['name']
)

first10

**Question 8: What are the 3 most popular movies?**


In [21]:
movie3 = cur.execute('SELECT original_title FROM movies ORDER by popularity DESC limit 3;').fetchall()
movie3

In [22]:
popular_movie = pd.DataFrame(cur.execute(
    'SELECT original_title, vote_average FROM movies ORDER by popularity DESC;').fetchall(),
    columns = ['movie','vote_average'])
popular_movie.head()

In [23]:
plt.figure(figsize=(15,5))
sns.barplot(popular_movie['vote_average'][:10],popular_movie['movie'][:10])
plt.title('Most popular 10 movies')
plt.show()

**Most expensive movies**

In [24]:
expensive = pd.DataFrame(
    cur.execute('SELECT original_title, name, release_date, budget, revenue FROM movies as M JOIN directors as D ON D.id = M.director_id ORDER BY budget DESC').fetchall(),
    columns = ['original_title','name','release_date','budget','revenue']
)
expensive[:10]


**Question 9: What are the 3 most bankable movies?**

In [25]:
expensive.original_title[:3]

**Most profit giving movies**

In [26]:
profit_giving = pd.DataFrame(cur.execute(
    'SELECT original_title, name, release_date, revenue FROM movies as M JOIN directors as D ON D.id = M.director_id ORDER BY revenue DESC').fetchall(),
    columns = ['original_title', 'director_name', 'release_date', 'revenue'])

profit_giving.head(10)

**Question 10: What is the most awarded average vote since the January 1st, 2000?**

In [27]:
vote = cur.execute('SELECT original_title, vote_average FROM movies WHERE release_date > \'2000-01-01\' ORDER by vote_average DESC LIMIT 1;').fetchall()
vote

**Question 11: Which movie(s) were directed by Brenda Chapman?**

In [28]:
brenda = cur.execute('SELECT original_title FROM movies as M JOIN directors as D ON D.id = M.director_id WHERE D.name = \'Brenda Chapman\';').fetchall()
brenda

**Question 12: Whose director made the most movies?**

In [29]:
most_movie = pd.DataFrame(cur.execute(
    'SELECT name, COUNT(name) FROM directors as D JOIN movies as M ON D.id = M.director_id GROUP BY director_id ORDER BY count(name) DESC;').fetchall(),
    columns = ['Name', 'Number of movies'])                      
most_movie[:5]

**Question 13: Whose director is the most bankable?**

In [30]:
most_bankable = cur.execute('SELECT name FROM directors as D JOIN movies as M ON D.id = M.director_id GROUP BY director_id ORDER BY sum(budget) DESC limit 1;').fetchall()
most_bankable

**Coorelation between variables**

In [31]:
cor = movies.corr()
sns.heatmap(cor,annot = True,linewidth = 1.5)